In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

!pip install --upgrade pip
!pip install accelerate>=0.16.0
!pip install bitsandbytes==0.39.0
!pip install click>=8.0.4
!pip install datasets>=2.10.0
!pip install deepspeed>=0.8.3
!pip install faiss-cpu==1.7.4
!pip install ipykernel==6.22.0
!pip install scipy==1.11.1
!pip install torch>=2.0.0
!pip install transformers==4.28.1
!pip install peft==0.3.0
!pip install pytest==7.3.2
#!pip install CUDA

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
  Using cached bitsandbytes-0.39.0-py3-none-any.whl.metadata (9.8 kB)
Using cached bitsandbytes-0.39.0-py3-none-any.whl (92.2 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.1
    Uninstalling bitsandbytes-0.43.1:
      Successfully uninstalled bitsandbytes-0.43.1


In [ ]:
#!python -m bitsandbytes
#!pip install --upgrade CUDA
#!pip install CUDA==122.0.0

In [ ]:
import os
import numpy as np
import pandas as pd
from typing import Any, Dict, List, Tuple, Union
from datasets import Dataset, load_dataset, disable_caching

disable_caching() ## disable huggingface cache

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import TextDataset
from transformers import pipeline

import torch
from torch.utils.data import Dataset, random_split
from transformers import TrainingArguments, Trainer
import accelerate
#import bitsandbytes

from IPython.display import Markdown

import gc

2024-07-22 03:37:46.082001: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 03:37:46.082067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 03:37:46.084148: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-22 03:37:47.516474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!pip install boto3

import json, boto3

model_id = "databricks/dolly-v2-12b"
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.model_max_length = 200000

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#gc.collect()

from datasets import load_dataset

dataframe = load_dataset("ammarnasr/the-stack-java-clean")

#training_set = dataframe['train'][:10000]
#testing_set = dataframe['test'][:1000]

In [ ]:
# Check for GPU
import torch
print("CUDA Available:", torch.cuda.is_available())

# Install BitsAndBytes
!pip install --upgrade bitsandbytes

# (Optional) Install specific PyTorch version
!pip install torch==1.13.1+cu113 torchvision==0.14.1+cu113 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu113

# Check installation
import bitsandbytes as bnb
print(bnb.__version__)


CUDA Available: True
  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl.metadata (2.2 kB)
Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.39.0
    Uninstalling bitsandbytes-0.39.0:
      Successfully uninstalled bitsandbytes-0.39.0
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
ERROR: Could not find a version that satisfies the requirement torch==1.13.1+cu113 (from versions: 1.11.0, 1.11.0+cu113, 1.12.0, 1.12.0+cu113, 1.12.1, 1.12.1+cu113, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1)
ERROR: No matching distribution found for torch==1.13.1+cu113
0.43.1


In [ ]:
gc.collect()
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/path/to/cuda/lib:/path/to/another/cuda/lib

from transformers import AutoModelForSequenceClassification
!pip install bitsandbytes-cuda111
# Initialize the Mistral model for sequence classification

#model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=3) this crashes colab for no reason now

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # use_cache=False,
    device_map="auto", #"balanced",
    load_in_8bit=True,
    torch_dtype=torch.float16
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/23.8G [00:00<?, ?B/s]

In [ ]:
print(model)
model.resize_token_embeddings(len(tokenizer))

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 5120)
    (layers): ModuleList(
      (0-35): 36 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear8bitLt(in_features=5120, out_features=15360, bias=True)
          (dense): Linear8bitLt(in_features=5120, out_features=5120, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=5120, out_features=20480, bias=True)
          (dense_4h_to_h): Linear8bitLt(in_features=20480, out_features=5120, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=5120, out_features=50280, bias=False)
)

Embedding(50280, 5120)

In [ ]:
from functools import partial
import copy

MAX_LENGTH = 256

def _preprocess_batch(batch: Dict[str, List]) -> Dict[str, Any]:
    model_inputs = tokenizer(batch["content"], max_length=MAX_LENGTH, truncation=True, padding='max_length')
    model_inputs["labels"] = copy.deepcopy(model_inputs['input_ids'])
    return model_inputs

_preprocessing_function = partial(_preprocess_batch)

In [ ]:
from datasets import Dataset
#gc.collect()
panda_dataframe = pd.concat([pd.DataFrame(dataframe[split]) for split in dataframe.keys()])

shortened_dataframe = panda_dataframe.head(1000)
panda_dataframe = panda_dataframe.sample(frac=1)

hf_dataset = Dataset.from_pandas(shortened_dataframe)

#panda_dataframe = pd.DataFrame(dataframe_train)
#dataframe_short = panda_dataframe.head(100)

gc.collect()

encoded_training_set = hf_dataset.map(
    _preprocessing_function,
    batched=True,
    batch_size=None,
    remove_columns=["hexsha", "size", "avg_line_length", "max_line_length", "alphanum_fraction"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
"""shortened_dataframe = panda_dataframe.head(1000)

encoded_training_set = []

for text in shortened_dataframe["content"]:
  model_inputs = tokenizer(text, max_length=MAX_LENGTH, truncation=True, padding='max_length')
  model_inputs["labels"] = copy.deepcopy(model_inputs['input_ids'])
  encoded_training_set.append(model_inputs)

encoded_training_set_dataframe = pd.DataFrame(encoded_training_set)"""
#processed_dataset = df.filter(lambda rec: len(rec["input_ids"]) <= MAX_LENGTH)

'shortened_dataframe = panda_dataframe.head(1000)\n\nencoded_training_set = []\n\nfor text in shortened_dataframe["content"]:\n  model_inputs = tokenizer(text, max_length=MAX_LENGTH, truncation=True, padding=\'max_length\')\n  model_inputs["labels"] = copy.deepcopy(model_inputs[\'input_ids\'])\n  encoded_training_set.append(model_inputs)\n\nencoded_training_set_dataframe = pd.DataFrame(encoded_training_set)'

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(encoded_training_set, test_size=0.2, random_state=42)

train_df = Dataset.from_dict(train_df)
test_df = Dataset.from_dict(test_df)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType, PeftConfig, PeftModel

MICRO_BATCH_SIZE = 8
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LORA_R = 8
LORA_ALPHA = 32
LORA_DROPOUT = 0.10


lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["query_key_value"],
)

In [ ]:
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 5898240 || all params: 11847792640 || trainable%: 0.0497834506327079


In [ ]:
gc.collect()

976

In [ ]:
from transformers import DataCollatorForSeq2Seq, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, pad_to_multiple_of=8, mlm=False)

#data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, pad_to_multiple_of=8, mlm=False, return_tensors='pt')

In [ ]:
EPOCHS = 8
LEARNING_RATE = 1e-4
MODEL_SAVE_FOLDER_NAME = "dolly12b-finetuned-True100000samples-Epoch5-lr-1e"

training_args = TrainingArguments(
                    output_dir=MODEL_SAVE_FOLDER_NAME,
                    overwrite_output_dir=True,
                    fp16=True,
                    per_device_train_batch_size=1,
                    per_device_eval_batch_size=1,
                    learning_rate=LEARNING_RATE,
                    # optim="adafactor",
                    num_train_epochs=EPOCHS,
                    logging_strategy="epoch",
                    evaluation_strategy="epoch",
                    save_strategy="no",
)

In [ ]:
%%time
trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=train_df,
        eval_dataset=test_df,
        data_collator=data_collator,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

gc.collect()

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/us

Epoch,Training Loss,Validation Loss
1,0.991600,0.966338
2,0.906500,0.973898
3,0.815800,1.007950
4,0.716000,1.045929
5,0.619300,1.097071
6,0.534100,1.155640
7,0.465000,1.196639
8,0.414500,1.245088


CPU times: user 6h 5min 20s, sys: 24.1 s, total: 6h 5min 44s
Wall time: 6h 4min 27s


TrainOutput(global_step=6400, training_loss=0.6828573274612427, metrics={'train_runtime': 21866.4815, 'train_samples_per_second': 0.293, 'train_steps_per_second': 0.293, 'total_flos': 1.13937865506816e+17, 'train_loss': 0.6828573274612427, 'epoch': 8.0})

In [ ]:



# /content/drive/fine-tuned-models/dolly

trainer.model.save_pretrained(MODEL_SAVE_FOLDER_NAME)

In [ ]:
trainer.save_model(MODEL_SAVE_FOLDER_NAME)
trainer.model.config.save_pretrained(MODEL_SAVE_FOLDER_NAME)

In [ ]:
import os

# List files in the saved model directory
os.listdir(MODEL_SAVE_FOLDER_NAME)

['tokenizer_config.json',
 'tokenizer.json',
 'adapter_model.bin',
 'adapter_config.json',
 'training_args.bin',
 'pytorch_model.bin',
 'config.json',
 'runs',
 'special_tokens_map.json']

In [ ]:
inference_prompt = "Can you make a Hello World program for me in Java?"

In [ ]:
%%capture
# Inference pipeline with the fine-tuned model
inf_pipeline =  pipeline('text-generation', model=trainer.model, tokenizer=tokenizer, max_length=256, trust_remote_code=True, do_sample=True)

# Format the prompt using the `prompt_template` and generate response
response = inf_pipeline(text_inputs=inference_prompt)[0]['generated_text']

response

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForCausalLM', 'RoCBer

In [ ]:
print(response)

Can you make a Hello World program for me in Java? Then you can join BugTraq.
import java.util.*;

public class test {
  public static void main(int i,String s,double d) {
    System.out.println("Hello World");
  }
}

Java is a very easy to learn programming language that I started my career with a few years back. This is one of the reasons many of the companies look at me for Java related jobs. And trust me, I have done Java related projects, my latest one is using React.js which is a JavaScript Front End framework.


Backend is in MySQL.


I recently joined Qualitrics as a Principal Software Engineer, building dashboad and mobile applications for our customers in the Aerospace and Embedded space.

I know C/C++, Java, Python. A little bit of shell scripting.

Would be glad to help you in your programming projects.

Cheers.


Nirav


In [ ]:
drive.mount('/content/drive/')
trainer.model.save_pretrained('/content/drive/MyDrive/fine-tuned-models/dolly')
trainer.save_model('/content/drive/MyDrive/fine-tuned-models/dolly')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
trainer.model.config.save_pretrained('/content/drive/MyDrive/fine-tuned-models/dolly')